# PyTorch Tutorial  

## Notes:  
1. The sequential model and the data should be assigned a device.  
    - model.to(device)  
    - data.to(device)  
2. For every batch, model.zero_grad() should be used to set the gradients to zero, as pytorch accumulated the gradients.  

In [5]:
%config IPCompleter.greedy=True
import torch
print(torch.cuda.is_available())

True


** Notes **  
1. **Autograd**  
    - 2 important classes: Tensor and Function  
    - Tensor and Function are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each variable has a .grad_fn attribute that references a Function that has created the Tensor 
    - For grad(), if its not a scalar, you need to specify a gradient argument that is a tensor of matching shape 
 

In [43]:
# Autograd basics
x = torch.ones(2, 2, requires_grad=True)
z = torch.ones(2, 2, requires_grad=True)
print('Since x is an independent variable, it has grad_fn =',x.grad_fn)
y = x**2 + z**3
print('Since y is dependent on x, it has grad_fn =',y.grad_fn)
print('backward prints dy/dx, so dy/dx=2x')
y.backward(gradient = z, retain_graph=True)
print(z.grad)
print('backward prints dy/dz, so dy/dz=3x')
y.backward(gradient = z)
print(z.grad)

Since x is an independent variable, it has grad_fn = None
Since y is dependent on x, it has grad_fn = <AddBackward1 object at 0x7fd8174f2048>
backward prints dy/dx, so dy/dx=2x
tensor([[ 3.,  3.],
        [ 3.,  3.]])
backward prints dy/dz, so dy/dz=3x
tensor([[ 6.,  6.],
        [ 6.,  6.]])


**Neural Nets**  
1. when we call loss.backward(), the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has requres_grad=True will have their .grad Tensor accumulated with the gradient.

In [ ]:
# Simple Neural Net
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
output = net(input)
target = torch.arange(1, 11)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


## convTranspose2D  

1. In pytorch, zero padding in convTranspose2d is done on the right and bottom sides only.  
2. With padding=1, 1 -> ((1,0),(0,0))  
3. **a1 = conv2d(b1), a2 = convTranspose2D(b2) ; if shapes of b2=a1, then shapes of a2=b1, ie., to know what will be the output shape of a convTranspose2D on X, just see doing conv2d on what shape would lead to the shape of X. **  

Examples for input and output of convTranspose2d:  

In [4]:
from torchsummary import summary
import torch.nn as nn
import torch

model = nn.Sequential(nn.ConvTranspose2d(100, 64, 4, 1, 0, bias=False))
noise = torch.randn(4, 100, 1, 1)
print('padding=0')
summary(model, (100,1,1))
print('\n')

model = nn.Sequential(nn.ConvTranspose2d(100, 64, 4, 1, 1, bias=False))
noise = torch.randn(4, 100, 1, 1)
print('padding=1')
summary(model, (100,1,1))
print('\n')

padding=0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1             [-1, 64, 4, 4]          102400
Total params: tensor(1.0240e+05)
Trainable params: tensor(1.0240e+05)
Non-trainable params: tensor(0)
----------------------------------------------------------------


padding=1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1             [-1, 64, 2, 2]          102400
Total params: tensor(1.0240e+05)
Trainable params: tensor(1.0240e+05)
Non-trainable params: tensor(0)
----------------------------------------------------------------


